In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold

import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score
from scipy.sparse import hstack



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/feedback-prize-english-language-learning/sample_submission.csv
../input/feedback-prize-english-language-learning/train.csv
../input/feedback-prize-english-language-learning/test.csv


In [65]:
train = pd.read_csv('../input/feedback-prize-english-language-learning/train.csv').fillna(' ')
test = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv').fillna(' ')


In [26]:
##  grammar,phraseology, vocabulary, syntax these targets imbalanced distribution.
imb = ["grammar","phraseology", "vocabulary", "syntax" ]

4.00    1096
4.00     988
4.00     790
4.00     534
4.00     315
4.00     125
4.00      27
4.00      26
4.00      10
Name: cohesion, dtype: int64

In [61]:
# Before log trans
from scipy import stats

for col in imb:
    _, p = stats.normaltest(train[col])
    print(col , p)
    
# grammar 3.409801751372249e-16
# phraseology 0.0006579263569791345
# vocabulary 3.7941570441109205e-11
# syntax 0.0046464626818017996

grammar 6.773897984120298e-184
phraseology 5.033632865701668e-159
vocabulary 3.0874887051342245e-221
syntax 1.8538677924859478e-199


In [68]:
for col in imb:
    train[col] = np.sqrt(train[col])

In [69]:
#after log trans
from scipy import stats

for col in imb:
    _, p = stats.normaltest(train[col])
    print(col , p)

grammar 3.8607624013934636e-10
phraseology 8.699522738101006e-09
vocabulary 8.421801102165705e-10
syntax 9.762503984593874e-08


In [28]:
class_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']


In [10]:
#for col in class_cols:
   # train[col] = train[col].astype("int")

In [12]:
train_text = train['full_text']
test_text = test['full_text']
all_text = pd.concat([train_text, test_text])

In [13]:
from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score  
filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer 
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)
pd.set_option("display.float_format", lambda x:"%.2f" % 4)
pd.set_option("display.max_rows", None)

In [14]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [16]:
import nltk
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [17]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [ ]:
#test_features = test_features.todense()

In [ ]:
#train_features = train_features.todense()

In [ ]:
columns = test.columns[1:]
columns

In [ ]:
from scipy.sparse import coo_matrix

In [18]:
df_subm = pd.DataFrame(columns=["text_id",'cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'])
df_subm["text_id"] = test["text_id"]


,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,NaN,NaN,NaN,NaN,NaN,NaN
1,000BAD50D026,NaN,NaN,NaN,NaN,NaN,NaN
2,00367BB2546B,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
X = train_features
X_test = test_features
y = train[['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']].values

In [20]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [22]:
X_train,X_temp, y_train, y_temp = train_test_split(X,y,test_size=0.2, random_state=0)


In [23]:

from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor

model_lgbm = MultiOutputRegressor(LGBMRegressor(learning_rate=0.05, boosting_type = "dart",colsample_bytree=0.9, num_iterations=1000, random_state=19, verbose=-1))

model_lgbm.fit(X_train, y_train)
    


KeyboardInterrupt: 

In [ ]:
pred = model_lgbm.predict(test_features)
preds = pd.DataFrame(pred, columns=class_cols)
submit_df = pd.concat([test[['text_id']], preds], axis=1)

In [ ]:
for col in imb:
    submit_df[col] = submit_df[col] * submit_df[col]

In [ ]:
submit_df.to_csv("submission.csv", index=False)